In [109]:
### 输出显著性块
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def generate_fixation_map(saliency_map, threshold=0.85, num_fixations=100, point_size=3):
    """
    从显著性图生成注视点图，使用较大的点来表示注视点，确保覆盖显著区域。

    参数:
    - saliency_map: 输入的显著性图，值在 0-255 之间的灰度图。
    - threshold: 阈值，用于限制注视点生成区域，值应在 0-1 之间。
    - num_fixations: 生成的注视点数量。
    - point_size: 注视点的大小，用于可视化。

    返回:
    - fixation_map: 注视点图，注视点为1，背景为0。
    """
    # 归一化显著性图到 [0, 1]
    saliency_map = saliency_map.astype(np.float32) / 255.0

    # 阈值处理，选择显著性高于阈值的区域
    high_saliency_indices = np.where(saliency_map >= threshold)

    # 如果没有满足阈值的区域，直接返回空白注视点图
    if len(high_saliency_indices[0]) == 0:
        return np.zeros_like(saliency_map)

    # 创建空白的注视点图
    fixation_map = np.zeros_like(saliency_map)

    # 确保显著区域都覆盖：在显著性较高的区域生成点
    for i in range(len(high_saliency_indices[0])):
        y, x = high_saliency_indices[0][i], high_saliency_indices[1][i]
        # 将选中的点扩展为一个小区域，例如 3x3 或 5x5
        fixation_map[max(0, y-point_size//2):min(fixation_map.shape[0], y+point_size//2+1),
                     max(0, x-point_size//2):min(fixation_map.shape[1], x+point_size//2+1)] = 1

    # 随机从高显著性区域中选择部分点作为主要注视点（可选）
    if len(high_saliency_indices[0]) > num_fixations:
        selected_indices = np.random.choice(len(high_saliency_indices[0]), num_fixations, replace=False)
        for idx in selected_indices:
            y, x = high_saliency_indices[0][idx], high_saliency_indices[1][idx]
            # 增大主要注视点的大小以更好地表示
            fixation_map[max(0, y-point_size//2):min(fixation_map.shape[0], y+point_size//2+1),
                         max(0, x-point_size//2):min(fixation_map.shape[1], x+point_size//2+1)] = 1

    return fixation_map

def process_images(input_folder, output_folder, threshold=0.15, num_fixations=100, point_size=1):
    """
    从输入文件夹读取所有图像文件，生成注视点图并保存到输出文件夹。

    参数:
    - input_folder: 输入图像文件夹路径。
    - output_folder: 输出图像文件夹路径。
    - threshold: 生成注视点图的显著性阈值。
    - num_fixations: 每张图生成的注视点数量。
    - point_size: 注视点的大小。
    """
    # 创建输出文件夹（如果不存在）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历输入文件夹中的所有文件
    for filename in os.listdir(input_folder):
        # 构建完整的文件路径
        input_path = os.path.join(input_folder, filename)
        
        # 确保只处理图像文件
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        # 读取显著性图（灰度图）
        saliency_map = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        # 生成注视点图
        fixation_map = generate_fixation_map(saliency_map, threshold=threshold, num_fixations=num_fixations, point_size=point_size)

        # 构建输出文件路径
        output_path = os.path.join(output_folder, filename)

        # 保存注视点图
        cv2.imwrite(output_path, fixation_map * 255)  # 乘以255以保存为8位图像

        # 可选：显示处理进度
        print(f'Processed and saved: {output_path}')

# 定义文件夹路径
#input_folder = 'datasets/AI4VA/train/train_saliency'
#output_folder = 'datasets/AI4VA/train/train_fixation'

#input_folder = 'datasets/AI4VA/val/val_saliency'
#output_folder = 'datasets/AI4VA/val/val_fixation'

# 定义文件夹2路径(双线程)
#input_folder = 'datasets1/AI4VA/train/train_saliency'
#output_folder = 'datasets1/AI4VA/train/train_fixation'

input_folder = 'datasets1/AI4VA/val/val_saliency'
output_folder = 'datasets1/AI4VA/val/val_fixation'

# 处理所有图像
process_images(input_folder, output_folder, threshold=0.3, num_fixations=100, point_size=1)


Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0471_1954_05_23-14.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0479_1954_07_18-01.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0480_1954_07_25-01.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0485_1954_08_29-01.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0525_1955_06_05-16.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0553_1955_12_18-06.png
Processed and saved: datasets1/AI4VA/val/val_fixation/Vaillant_0608_1957_01_06-06.png


In [84]:
# step1 根据预测图生成注视点
import cv2
import numpy as np
import csv
import os

# 读取显著性预测图
def load_saliency_map(file_path):
    return cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

# 应用阈值化处理
def apply_threshold(saliency_map):
    _, binary_map = cv2.threshold(saliency_map, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_map

# 在显著性高的地方生成更多注视点
def generate_dense_fixations(saliency_map, num_points=1500):
    height, width = saliency_map.shape
    fixations = []
    
    # 将显著性图像的值归一化，作为概率分布
    normalized_map = saliency_map.astype(float) / 255.0
    ############################################################
    # 确保归一化后的值在 [0, 1] 之间
    normalized_map = np.clip(normalized_map, 0, 1)
    
    # 防止概率分布和为0
    if normalized_map.sum() == 0:
        raise ValueError("Normalized saliency map has no valid values. Sum of values is zero.")
    ############################################################
    normalized_map /= normalized_map.sum()  # 确保概率总和为1

    # 将显著性图像展平为一维数组，方便使用np.random.choice生成注视点
    flat_map = normalized_map.flatten()
    
    # 生成(num_points)个注视点，根据显著性图中的像素值作为概率
    indices = np.random.choice(height * width, num_points, p=flat_map)

    # 将一维索引转换为二维的(x, y)坐标
    for idx in indices:
        x = idx % width
        y = idx // width
        fixations.append((x, y))
    
    return fixations

# 计算每个注视点的持续时间
def calculate_fixation_times(saliency_map, fixations):
    fixation_times = [saliency_map[y, x] / 255.0 for x, y in fixations]
    return fixation_times

# 保存注视点及其时间到CSV文件
def save_fixations_to_csv(file_path, fixations, fixation_times):
    with open(file_path, 'w', newline='') as csvfile:
        fieldnames = ['x', 'y', 'duration']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for (x, y), duration in zip(fixations, fixation_times):
            writer.writerow({'x': x, 'y': y, 'duration': duration})

# 绘制注视点并保存为新图像
def draw_fixations(fixations, output_image_path, image_size):
    # 创建一个黑色背景的新图像
    image_with_fixations = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    
    # 绘制注视点（小的白色圆点）
    for (x, y) in fixations:
        cv2.circle(image_with_fixations, (x, y), 2, (255, 255, 255), -1)  # 白色圆点，半径为2
    
    # 保存图像
    cv2.imwrite(output_image_path, image_with_fixations)

# 处理单个文件的函数
def process_image(saliency_map_path, output_image_folder, output_csv_folder, num_fixations=500):
    # 获取图像的文件名（不带扩展名）
    base_name = os.path.splitext(os.path.basename(saliency_map_path))[0]
    
    # 生成输出文件路径
    fixation_output_csv_path = os.path.join(output_csv_folder, f'{base_name}_fixations.csv')
    fixation_output_image_path = os.path.join(output_image_folder, f'{base_name}.png')
    
    # 读取显著性预测图
    saliency_map = load_saliency_map(saliency_map_path)
    
    # 应用阈值化处理
    saliency_map = apply_threshold(saliency_map)
    
    # 生成密集的注视点，在显著性高的区域生成更多注视点
    fixations = generate_dense_fixations(saliency_map, num_points=num_fixations)
    
    # 计算每个注视点的持续时间
    fixation_times = calculate_fixation_times(saliency_map, fixations)
    
    # 保存注视点及其时间到CSV文件
    save_fixations_to_csv(fixation_output_csv_path, fixations, fixation_times)
    
    # 绘制注视点并保存为新图像
    image_size = saliency_map.shape
    draw_fixations(fixations, fixation_output_image_path, image_size)
    
    # print(f'Fixations saved to {fixation_output_csv_path}')
    print(f'Fixations image saved to {fixation_output_image_path}')

# 主函数，批处理文件夹中的所有图像
def main():
    input_folder = 'datasets/AI4VA/train/train_saliency'
    # input_folder = 'datasets/AI4VA/val/val_saliency'
    # 输出文件夹路径
    output_image_folder = 'datasets/AI4VA/train/train_fixation'
    # output_image_folder = 'datasets/AI4VA/val/val_fixation'
    
    output_csv_folder = 'output_csv_val'
    
    # 确保输出文件夹存在
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_csv_folder, exist_ok=True)
    
    # 批处理输入文件夹中的所有图像文件
    for filename in os.listdir(input_folder):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):  # 可根据需要添加更多扩展名
            saliency_map_path = os.path.join(input_folder, filename)
            process_image(saliency_map_path, output_image_folder, output_csv_folder)

if __name__ == "__main__":
    main()


Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0460_1954_03_07-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0466_1954_04_18-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0470_1954_05_16-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0470_1954_05_16-14.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0473_1954_06_06-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0476_1954_06_27-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0477_1954_07_04-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0479_1954_07_18-14.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0484_1954_08_22-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_0489_1954_09_26-01.png
Fixations image saved to datasets/AI4VA/train/train_fixation/Vaillant_

In [101]:
# 将热力图变为黑白色图像
import cv2
import numpy as np
import os

def convert_heatmap_to_bw(heatmap_path, output_path):
    # 读取热力图像
    heatmap = cv2.imread(heatmap_path)

    # 检查图像是否成功加载
    if heatmap is None:
        print(f"Failed to load image {heatmap_path}")
        return

    # 转换为灰度图像
    gray_heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2GRAY)

    # 归一化到 [0, 255]，并转换为无符号 8 位整数
    normalized_gray = cv2.normalize(gray_heatmap, None, 0, 255, cv2.NORM_MINMAX)

    # 保存黑白显著性图像
    cv2.imwrite(output_path, normalized_gray)

    print(f"Converted {heatmap_path} to black and white and saved as {output_path}")

# 示例使用
input_folder = "SUM_newK8"  # 热力图文件夹
output_folder = "SUM_newK8/SUM_newK8_heibai"  # 输出文件夹

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 处理文件夹中的所有图像
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # 根据图像格式修改
        heatmap_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder,filename)
        convert_heatmap_to_bw(heatmap_path, output_path)


Converted SUM_newK8/Vaillant_0471_1954_05_23-14.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0471_1954_05_23-14.png
Converted SUM_newK8/Vaillant_0479_1954_07_18-01.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0479_1954_07_18-01.png
Converted SUM_newK8/Vaillant_0480_1954_07_25-01.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0480_1954_07_25-01.png
Converted SUM_newK8/Vaillant_0485_1954_08_29-01.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0485_1954_08_29-01.png
Converted SUM_newK8/Vaillant_0525_1955_06_05-16.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0525_1955_06_05-16.png
Converted SUM_newK8/Vaillant_0553_1955_12_18-06.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0553_1955_12_18-06.png
Converted SUM_newK8/Vaillant_0608_1957_01_06-06.png to black and white and saved as SUM_newK8/SUM_newK8_heibai/Vaillant_0608_1957_01_06-06.png

In [104]:
import cv2
import os

# 输入与输出文件夹路径
input_folder = 'SUM_newK8/SUM_newK8_heibai/'  # 替换为你的输入文件夹路径
output_folder = 'SUM_newK8/SUM_newK8_gaosi23/'  # 替换为你的输出文件夹路径

# 检查输出文件夹是否存在，不存在则创建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 设置高斯模糊的核大小 (kernel size)，值越大模糊越明显，必须是奇数
kernel_size = (23, 23)

# 遍历输入文件夹中的所有图片
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".bmp"):  # 仅处理指定格式的图片
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 以灰度模式读取图片

        # 对图像进行高斯模糊处理
        blurred_img = cv2.GaussianBlur(img, kernel_size, 0)

        # 保存处理后的图像到输出文件夹
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, blurred_img)

print("处理完成，模糊后的图像已保存到输出文件夹。")


处理完成，模糊后的图像已保存到输出文件夹。


In [24]:
#根据文件制作csv文件
import os
import csv

def generate_train_ids_csv(input_folder, output_csv_path):
    # 打开CSV文件进行写入
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['image', 'map', 'fixation']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # 写入表头
        writer.writeheader()
        
        # 遍历文件夹中的所有图片
        for filename in os.listdir(input_folder):
            if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
                # 假设原始图像名是 filename，显著性图和注视点文件都以相同的基础名命名，只是后缀不同
                base_name = os.path.splitext(filename)[0]
                
                # 构建每列的文件名
                image_name = filename  # 原始图像名
                map_name = f"{base_name}.png" 
                fixation_name = f"{base_name}.png"  # 假设fixation文件名后缀为_fixations.png
                
                # 写入CSV文件
                writer.writerow({'image': image_name, 'map': map_name, 'fixation': fixation_name})

    print(f"CSV file saved to {output_csv_path}")

# 主函数
def main():
    # 输入图像文件夹
    input_folder = 'img_val'  # 替换为你的文件夹路径
    # 输出CSV文件路径
    output_csv_path = 'val_id.csv'
    
    # 生成CSV文件
    generate_train_ids_csv(input_folder, output_csv_path)

if __name__ == "__main__":
    main()


CSV file saved to val_id.csv


In [31]:
#修改图片大小为512
import os
from PIL import Image

# 图像文件夹路径
input_folder = 'data'
output_folder = 'data/resize'

# 创建输出文件夹（如果不存在的话）
os.makedirs(output_folder, exist_ok=True)

# 目标图像大小
target_size = (512, 512)

# 遍历输入文件夹中的所有图片并调整大小
def resize_images(input_folder, output_folder, target_size):
    for filename in os.listdir(input_folder):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            # 构建完整的输入图像路径
            img_path = os.path.join(input_folder, filename)
            
            # 打开图像并调整大小
            img = Image.open(img_path)
            img_resized = img.resize(target_size, Image.Resampling.LANCZOS)  # 使用LANCZOS替代ANTIALIAS
            
            # 构建输出图像路径
            output_img_path = os.path.join(output_folder, filename)
            
            # 保存调整后的图像
            img_resized.save(output_img_path)
            
            print(f"Processed {filename}")

# 批处理所有图像
resize_images(input_folder, output_folder, target_size)

print("All images have been resized.")


Processed Vaillant_0471_1954_05_23-14.png
Processed Vaillant_0479_1954_07_18-01.png
Processed Vaillant_0480_1954_07_25-01.png
Processed Vaillant_0485_1954_08_29-01.png
Processed Vaillant_0525_1955_06_05-16.png
Processed Vaillant_0553_1955_12_18-06.png
Processed Vaillant_0608_1957_01_06-06.png
All images have been resized.


In [26]:
# 讲指定文件夹中的文件从256*256变为512*512
import os
from PIL import Image

def resize_images_in_folder(folder_path, new_size=(512, 512)):
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        # 构建完整文件路径
        img_path = os.path.join(folder_path, filename)
        
        # 检查文件是否为图片
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            try:
                # 打开图片
                with Image.open(img_path) as img:
                    # 检查图片大小是否是256x256
                    if img.size == (256, 256):
                        # 将图片大小调整为512x512
                        resized_img = img.resize(new_size)
                        # 保存图片，覆盖原图
                        resized_img.save(img_path)
                        # print(f"Resized {filename} to {new_size}")
                    else:
                        print(f"Skipping {filename}: Image size is {img.size}, not 256x256")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

folder_path = 'datasets/datasets_UI_256/val/val_fixation'
resize_images_in_folder(folder_path)
print("finished")

finished
